# Проект: ВШЭ ПСБ.Хак

## Описание задачи


После суровых трудовых будней всегда хочется с удовольствием провести свой отпуск и насладиться отдыхом. Тем не менее, на этом пути к счастью обычно возникают несколько преград: выбор желаемой гостиницы, заблаговременное бронирование, оплата, получение максимально возможной скидки и т.д. Как минимизировать свои риски и траты, и при этом получить желаемый уровень сервиса и отдых?

А что, если проанализировать данные по оплатам, бронированиям и отменам для сети гостиниц и попытаться найти скрытые зависимости с датами заезда, формой оплаты и другими признаками?

А если на этом не останавливаться и попытаться использовать найденные паттерны для создания нового банковского продукта, который сможет как помочь потенциальным клиентам с организацией отпуска, так и позволит банку и отелю остаться в плюсе?

Именно для создания такого инновационного продукта Вышка Онлайн совместно с Цифровой лабораторией ПСБ проводит хакатон ВШЭ ПСБ.Хак.


### Организация


Соревнование пройдет с 20-22 сентября 2024 в гибридном формате. Командам участникам необходимо пройти через два этапа:

* **Этап 1.** Технический. Решение задачи бинарной классификации с целью предсказания факта отмены бронирования
* **Этап 2.** Бизнесовый. На основе найденных зависимостей в рамках первого этапа, а также с учетом самостоятельно проведенного анализа рынка и существующих банковских и онлайн-туристических продуктов и программ лояльности, предложить новый банковский продукт для банка ПСБ.
В рамках первого этапа команды подгружают свои предсказания отмены бронирования на платформу, где происходит автоматическая проверка ответов на тестовой выборке. Первый этап соревнования завершается 22 сентября в 16:00.

Детали регистрации на платформе и особенности отправки решений будут подробно описаны после торжественного открытия хакатона 20 сентября.

В рамках второго этапа команды готовят презентацию предлагаемого продукта, которую будут защищать перед жюри 22 сентября, начиная с 17:00.

Для погружения участников в особенности банковского бизнеса и предметной области в течение субботы будет проводится серия мастер-классов. Отдельно с командами будут работать менторы, которые будут консультировать на протяжении всего периода соревнования.

### Данные
Данные представляют собой информацию о бронировании номеров различных категорий в разных гостиницах. Гостиницы с номерами 1 и 2 – один регион РФ, с номерами 3 и 4 – другой регион РФ. Структура данных следующая:



* № брони – идентификатор брони
* Номеров – количество номеров в бронировании
* Стоимость – стоимость номеров в рублях
* Внесена предоплата – сумма внесенной предоплаты
* Способ оплаты – один из 12 способов оплаты
* Дата бронирования – дата бронирования с точностью до минуты
* Дата отмены – дата отмены бронирования с точностью до минуты, если было
* Заезд – дата заезда с точностью до дня
* Ночей – количество ночей
* Выезд – дата выезда с точностью до дня
* Источник – онлайн-канал продаж
* Статус брони – один из 5 статусов
* Категория номера – описание категории номера. Если бронировалось несколько номеров, то идет сплошное описание с * нумерацией.
* Гостей – число гостей
* Гостиница – номер гостиницы

* Целевое поле – Дата отмены: если поле заполнено, то это соответствует целевому значению 1, иначе – 0. Статусы в поле Статус брони уточняют состояние и носят справочный характер (но могут помочь при построении модели).

В тестовой выборке отсутствуют столбцы Дата отмены и Статус брони.

### Метрика
Итоговой метрикой выступает ROC-AUC по столбцу Дата отмены.


### Лидерборд
Лидерборд технической части соревнования (метрика модели) в процессе соревнования считается по доступной для скачивания тестовой выборке. Окончание отправок всех решений 22 сентября в 16:00. Лидерборд скрывается от участников 22 сентября в 12:00 и открывается с итоговым результатом 22 сентября в 20:00.

### Итоговая оценка решения
Итоговая оценка соревнования формируется на основе технической части (40% результата) и защит решений в форме презентаций (60% результата). Экспертное жюри определяет оценку защит. Детали с критериями оценки второго этапа будут предоставлены командам отдельно.

## Загрузка данных

### Импорт библиотек

In [4]:
import pandas as pd
import requests
from io import BytesIO
import os

### Функции

In [5]:
def github(url):
    response = requests.get(url)
    response.raise_for_status() 
    return BytesIO(response.content)

In [6]:
# Функция для загрузки файла из локальной папки или с GitHub
def load_f(file_path, github_url, file_type="xlsx"):
    try:

        if file_type == "xlsx":
            data = pd.read_excel(file_path)
        elif file_type == "csv":
            data = pd.read_csv(file_path)
        print(f"Файл загружен локально: {file_path}")

    except FileNotFoundError:
        print(f"Файл не найден локально, загружаю с GitHub: {github_url}")
        if file_type == "xlsx":
            data = pd.read_excel(github(github_url))
        elif file_type == "csv":
            data = pd.read_csv(github(github_url))
        print(f"Файл загружен с GitHub: {github_url}")

    return data

In [7]:
# Пути к локальным файлам и ссылки на GitHub
train_local_path = os.path.join('datasets', 'train.xlsx')
test_local_path = os.path.join('datasets', 'test.xlsx')
example_local_path = os.path.join('datasets', 'example.csv')

train_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/train.xlsx?raw=true"
test_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/test.xlsx?raw=true"
example_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/example.csv?raw=true"

# Загрузка файлов
train_data = load_f(train_local_path, train_github_url, file_type="xlsx")
test_data = load_f(test_local_path, test_github_url, file_type="xlsx")
example_data = load_f(example_local_path, example_github_url, file_type="csv")

Файл загружен локально: datasets\train.xlsx
Файл загружен локально: datasets\test.xlsx
Файл загружен локально: datasets\example.csv


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26174 entries, 0 to 26173
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          26174 non-null  int64         
 1   № брони             26174 non-null  object        
 2   Номеров             26174 non-null  int64         
 3   Стоимость           26174 non-null  float64       
 4   Внесена предоплата  26174 non-null  int64         
 5   Способ оплаты       26174 non-null  object        
 6   Дата бронирования   26174 non-null  datetime64[ns]
 7   Дата отмены         5192 non-null   datetime64[ns]
 8   Заезд               26174 non-null  datetime64[ns]
 9   Ночей               26174 non-null  int64         
 10  Выезд               26174 non-null  datetime64[ns]
 11  Источник            26174 non-null  object        
 12  Статус брони        26174 non-null  object        
 13  Категория номера    26174 non-null  object    

In [17]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11218 entries, 0 to 11217
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          11218 non-null  int64         
 1   № брони             11218 non-null  object        
 2   Номеров             11218 non-null  int64         
 3   Стоимость           11218 non-null  float64       
 4   Внесена предоплата  11218 non-null  int64         
 5   Способ оплаты       11218 non-null  object        
 6   Дата бронирования   11218 non-null  datetime64[ns]
 7   Заезд               11218 non-null  datetime64[ns]
 8   Ночей               11218 non-null  int64         
 9   Выезд               11218 non-null  datetime64[ns]
 10  Источник            11218 non-null  object        
 11  Категория номера    11218 non-null  object        
 12  Гостей              11218 non-null  int64         
 13  Гостиница           11218 non-null  int64     

In [18]:
example_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11217 entries, 0 to 11216
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       11217 non-null  int64
dtypes: int64(1)
memory usage: 87.8 KB
